In [1]:
import argparse
import os

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchaudio

import hydra
from omegaconf import OmegaConf
from torch.distributions import Categorical
from tqdm.auto import tqdm

from src import utils
from src.dataloaders.audio import mu_law_decode
from src.models.baselines.wavenet import WaveNetModel
from train import SequenceLightningModule

import shap
import scipy as sp
import matplotlib.pyplot as plt

/home/ys724/.conda/envs/vs4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run -m generate experiment=lm/s4-wt103 checkpoint_path=checkpoints/s4-wt103.pt n_samples=1 l_sample=10 l_prefix=5 decode=text

CONFIG
├── train
│   └── seed: 1111                                                                                                 
│       name: null                                                                                                 
│       interval: step                                                                                             
│       monitor: val/loss                                                                                          
│       mode: min                                                                                                  
│       ema: 0.0                                                                                                   
│       test: false                                                                                                
│       debug: false                                                                                               
│       ignore_warnings: false                                                                                     
│       state:                                                                                                     
│         mode: null                                                                                               
│         n_context: 0                                                                                             
│         n_context_eval: 0                                                                                        
│       ckpt: null                                                                                                 
│       disable_dataset: false                                                                                     
│       validate_at_start: false                                                                                   
│       pretrained_model_path: null                                                                                
│       pretrained_model_strict_load: true                                                                         
│       pretrained_model_state_hook:                                                                               
│         _name_: null                                                                                             
│       post_init_hook:                                                                                            
│         _name_: null                                                                                             
│       layer_decay:                                                                                               
│         _name_: null                                                                                             
│         decay: 0.7                                                                                               
│                                                                                                                  
├── tolerance
│   └── logdir: ./resume                                                                                           
│       id: null                                                                                                   
│                                                                                                                  
├── wandb
│   └── project: hippo                                                                                             
│       group: ''                                                                                                  
│       job_type: training                                                                                         
│       mode: online                                                                                               
│       save_dir: .                                                                                                
│       id: s4-

CONFIG
├── train
│   └── seed: 1111                                                                                                 
│       name: null                                                                                                 
│       interval: step                                                                                             
│       monitor: val/loss                                                                                          
│       mode: min                                                                                                  
│       ema: 0.0                                                                                                   
│       test: false                                                                                                
│       debug: false                                                                                               
│       ignore_warnings: false                                                                                     
│       state:                                                                                                     
│         mode: null                                                                                               
│         n_context: 0                                                                                             
│         n_context_eval: 0                                                                                        
│       ckpt: null                                                                                                 
│       disable_dataset: false                                                                                     
│       validate_at_start: false                                                                                   
│       pretrained_model_path: null                                                                                
│       pretrained_model_strict_load: true                                                                         
│       pretrained_model_state_hook:                                                                               
│         _name_: null                                                                                             
│       post_init_hook:                                                                                            
│         _name_: null                                                                                             
│       layer_decay:                                                                                               
│         _name_: null                                                                                             
│         decay: 0.7                                                                                               
│                                                                                                                  
├── tolerance
│   └── logdir: ./resume                                                                                           
│       id: null                                                                                                   
│                                                                                                                  
├── wandb
│   └── project: hippo                                                                                             
│       group: ''                                                                                                  
│       job_type: training                                                                                         
│       mode: online                                                                                               
│       save_dir: .                                                                                                
│       id: s4-

[rank: 0] Global seed set to 1111


Loading model...
Full checkpoint path: /home/ys724/S4/State-Space-Interpretability/state-spaces/checkpoints/s4-wt103.pt
[2023-05-05 22:19:43,996][root][INFO] - Loading cached dataset...
Vocab size: 267735
[2023-05-05 22:19:46,056][src.models.sequence.kernels.ssm][WARNING] - CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
[2023-05-05 22:19:46,970][src.models.sequence.kernels.ssm][INFO] - Pykeops installation found.
[2023-05-05 22:19:46,986][src.models.sequence.kernels.ssm][INFO] - Constructing S4 (H, N, L) = (1024, 32, 8192)
[2023-05-05 22:19:47,009][src.models.sequence.kernels.ssm][INFO] - Constructing S4 (H, N, L) = (1024, 32, 8192)
[2023-05-05 22:19:47,097][src.models.sequence.kernels.ssm][INFO] - Constructing S4 (H, N, L) = (1024, 32, 8192)
[2023-05-0

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 55.20it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 56.36it/s]


shap forward pass done with output torch.Size([1, 10]) (1,)
x ['<eos>' '=' 'Homarus' 'gammarus' '=']
mask [False False False False False]
x_tensor tensor([     0,      9,  96220, 198382,      9])
output tensor([[0, 0, 0, 0, 0]])
explainer start with input [[0 0 0 0 0]]


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 57.46it/s]


shap forward pass done with output torch.Size([1, 10]) (1,)
permutation explainer


!!!!!!!!!!


In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import shap
import torch

/home/ys724/.conda/envs/vs4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("gpt2").cuda()
# set model decoder to true
model.config.is_decoder=True
# set text-generation params under task_specific_params
model.config.task_specific_params["text-generation"] = {
    "do_sample": True,
    "max_length": 50,
    "temperature": 0.7,
    "top_k": 50,
    "no_repeat_ngram_size": 2
}
s = ['I enjoy walking with my cute dog']

In [4]:
explainer = shap.Explainer(model, tokenizer)
shap_values = explainer(s)
print(shap_values)
# .values [1, input_len, output_len]
# .base_values [1, output_len]
# .data = [output_len] - str
shap.plots.text(shap_values)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.


.values =
array([[[-1.16613934e-01,  8.72616787e-01,  1.00514289e+00,
          1.25749471e-01,  1.99217282e-01, -4.10349673e-01,
         -6.36086927e-02,  4.05762883e-01, -1.30096765e-01,
          1.27657733e-02,  7.76433187e-02, -3.32962458e-01,
          8.88581558e-02],
        [-4.31012734e-01,  1.01470339e+00,  2.32480506e-02,
         -1.58169040e-01,  1.67172922e-01,  1.49489320e-01,
          3.62267154e-02,  3.55961711e-01,  4.57240805e-01,
          1.14920521e-02,  1.59498516e-02, -2.32643776e-02,
          5.82410425e-02],
        [-4.26616845e-01, -2.93118360e-02, -1.94200333e-03,
         -3.55014850e-02, -1.96080540e-01, -3.22592551e-01,
          1.87846905e-01,  1.71433277e-01, -4.59277155e-02,
          2.31572030e-01,  3.91687669e-01,  2.02801050e-01,
          1.70823823e-01],
        [ 7.23258793e-02,  3.10825985e-01, -1.06642422e-01,
         -8.20662956e-02,  1.83227621e-02,  4.08413111e-01,
          5.25015842e-02, -9.38722999e-02, -4.91522092e-03,
         

!!!!!!!!!!
